<a href="https://colab.research.google.com/github/varunvenkatsaiis/colab_notebooks/blob/main/seq2_seq_rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip
!unzip -q spa-eng.zip

--2025-05-18 14:39:30--  http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.101.207, 142.250.141.207, 142.251.2.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.101.207|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2638744 (2.5M) [application/zip]
Saving to: ‘spa-eng.zip.1’

spa-eng.zip.1       100%[===================>]   2.52M  --.-KB/s    in 0.007s  

2025-05-18 14:39:30 (346 MB/s) - ‘spa-eng.zip.1’ saved [2638744/2638744]

replace spa-eng/_about.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace spa-eng/spa.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y


In [ ]:
text_file = "spa-eng/spa.txt"

In [ ]:
with open(text_file) as f:
  lines = f.read().split("\n")[:-1]

text_pairs = []

for line in lines:
  english, spanish = line.split("\t")
  spanish = "[start] " + spanish + " [end]"
  text_pairs.append( (english, spanish) )

In [ ]:
import random

random.shuffle(text_pairs)

num_val_samples = int(0.15 * len(text_pairs))

num_train_samples = len(text_pairs) - 2 * (num_val_samples)

train_pairs  = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples : num_train_samples + num_val_samples]

test_pairs = text_pairs[ num_train_samples + num_val_samples : ]

In [ ]:
import tensorflow as tf
import string
from tensorflow import keras
from tensorflow.keras.layers import  TextVectorization

In [ ]:
strip_chars = string.punctuation + "¿"
strip_chars = strip_chars.replace("[" ,"")
strip_chars = strip_chars.replace("]" ,"")



In [ ]:
def custom_standardization(input_string):
  lowercase = tf.strings.lower(input_string)
  return tf.strings.regex_replace(lowercase , f"[{re.escape(strip_chars)}]" , "")


In [ ]:
vocab_size = 15000
sequence_length = 20

source_vectorization = TextVectorization(
    max_tokens = vocab_size,
    output_mode = "int",
    output_sequence_length = sequence_length,
)

target_vectorization = TextVectorization(
    max_tokens = vocab_size,
    output_mode = "int",
    output_sequence_length = sequence_length + 1,
    standardize = custom_standardization,
)

train_english_texts = [ pair[0] for pair in train_pairs ]
train_spanish_texts = [ pair[1] for pair in train_pairs ]

source_vectorization.adapt(train_english_texts)
target_vectorization.adapt(train_spanish_texts)

In [ ]:
batch_size = 64

def format_dataset(eng , spa):
  eng = source_vectorization(eng)
  spa = target_vectorization(spa)
  return ({
      "english" : eng,
      "spanish" : spa[: , :-1],
  } , spa[: , 1:])

def make_dataset(pairs):
  eng_texts , spa_texts = zip(*pairs)
  eng_texts = list(eng_texts)
  spa_texts = list(spa_texts)
  dataset = tf.data.Dataset.from_tensor_slices((eng_texts , spa_texts))
  dataset = dataset.batch(batch_size)
  dataset = dataset.map(format_dataset , num_parallel_calls=4)
  return dataset.shuffle(2048).prefetch(16).cache()

In [ ]:
train_ds = make_dataset(train_pairs)
test_ds = make_dataset( test_pairs)


In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

embed_dim = 256
latent_dim = 1024

source = keras.Input(shape=(None ,) ,dtype="int64" , name = "english")
x = layers.Embedding(vocab_size ,embed_dim , mask_zero=True)(source)

encoded_source = layers.Bidirectional(  layers.GRU(latent_dim) , merge_mode="sum" )(x)

In [ ]:
past_target = keras.Input( shape=(None , ) , dtype="int64" , name = "spanish" )

x = layers.Embedding(vocab_size , embed_dim , mask_zero=True ) (past_target)

decoder_gru = layers.GRU(latent_dim , return_sequences=True)

x = decoder_gru(x , initial_state=encoded_source)

x = layers.Dropout(0.5)(x)

target_next_step = layers.Dense(vocab_size , activation="softmax")(x)

seq2seq_rnn = keras.Model([source , past_target] , target_next_step)

